# CASA0013 Data and Policy
Investigating the relationship between Airbnb and crime in London

## Import packages

In [1]:
import numpy as np
import pandas as pd

## Reading in the data
Datasets to be used for this analysis
- Airbnb data from Inside Airbnb
- Crime data 

In [2]:
# Set download URL
host = 'https://orca.casa.ucl.ac.uk' # where the data is stored
path = '~jreades/data/2022-09-10-listings.csv.gz' # compressed using gzip
url  = f'{host}/{path}'

# your code here
df = pd.read_csv(url, compression='gzip', low_memory=False)
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 69,351 x 75


In [4]:
crime_url = "https://data.london.gov.uk/download/recorded_crime_summary/644a9e0d-75a3-4c3a-91ad-03d2a7cb8f8e/MPS%20LSOA%20Level%20Crime%20%28most%20recent%2024%20months%29.csv"
crime = pd.read_csv(crime_url)
print(f"Data frame is {crime.shape[0]:,} x {crime.shape[1]}")

Data frame is 102,755 x 29


In [6]:
crime.head(5)

,LSOA Code,LSOA Name,Borough,Major Category,Minor Category,202011,202012,202101,202102,202103,...,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210
0,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Arson and Criminal Damage,Criminal Damage,0,0,1,1,0,...,2,1,0,0,0,0,1,1,0,0
1,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Burglary,Burglary Business and Community,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Burglary,Domestic Burglary,4,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Drug Offences,Drug Trafficking,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,E01000006,Barking and Dagenham 016A,Barking and Dagenham,Drug Offences,Possession of Drugs,0,0,0,2,3,...,5,2,1,1,2,0,0,1,0,4


In [8]:
crime_historic_url = "https://data.london.gov.uk/download/recorded_crime_summary/9bb3efe6-98fb-47c1-af3e-3be721e8a1cd/MPS%20LSOA%20Level%20Crime%20%28Historical%29.csv"
crime_historic = pd.read_csv(crime_historic_url)
print(f"Data frame is {crime_historic.shape[0]:,} x {crime_historic.shape[1]}")

Data frame is 146,354 x 132


In [11]:
lsoa_atlas_url = "https://data.london.gov.uk/download/lsoa-atlas/0193f884-2ccd-49c2-968e-28aa3b1c480d/lsoa-data.csv"
lsoa_atlas = pd.read_csv(lsoa_atlas_url, encoding = 'unicode_escape')
print(f"Data frame is {lsoa_atlas.shape[0]:,} x {lsoa_atlas.shape[1]}")

Data frame is 4,837 x 277


/tmp/ipykernel_110/2548509382.py:2: DtypeWarning: Columns (174,175,176,178,189,253,254) have mixed types. Specify dtype option on import or set low_memory=False.
  lsoa_atlas = pd.read_csv(lsoa_atlas_url, encoding = 'unicode_escape')
